In [1]:
import logging
import tempfile
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, CompositeVideoClip, ColorClip, AudioFileClip
from pytubefix import YouTube
import numpy as np
import cv2
from google.cloud import videointelligence
import io
import time
import json
import logging
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
import os

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [ ]:
"https://www.youtube.com/watch?v=aKq8bkY5eTU" == "https://www.youtube.com/watch?v=aKq8bkY5eTU"

In [2]:
url = "https://www.youtube.com/watch?v=aKq8bkY5eTU"
logging.info(f"Downloading video from {url}...")

# Create a temporary directory to store the video
# temp_dir = tempfile.mkdtemp()
temp_dir = os.path.abspath(os.getcwd())

2025-04-03 14:04:45,237 - INFO - Downloading video from https://www.youtube.com/watch?v=aKq8bkY5eTU...


In [3]:
if "youtu.be/" in url:
    video_id = url.split("youtu.be/")[1].split("?")[0]
    url = f"https://www.youtube.com/watch?v={video_id}"
url

'https://www.youtube.com/watch?v=aKq8bkY5eTU'

In [5]:
yt = YouTube(url)

# Add an event handler for on_progress_callback
yt.register_on_progress_callback(lambda stream, chunk, bytes_remaining: 
                    logging.debug(f"Downloaded {stream.filesize - bytes_remaining} of {stream.filesize} bytes"))

# Get the highest quality progressive stream
streams = yt.streams.filter(progressive=True, file_extension='mp4')
if not streams:
    raise ValueError("No suitable video streams found for this YouTube video.")
    
video = streams.order_by('resolution').desc().first()

if not video:
    raise ValueError("Could not find a suitable video format to download.")

In [7]:
print(yt.captions)
try: 
    caption = yt.captions['a.en']
except:
    caption = yt.captions['a.hi']

cap = caption.generate_srt_captions()
print(cap)

{'ar': <Caption lang="Arabic" code="ar">, 'bn': <Caption lang="Bangla" code="bn">, 'zh-Hant': <Caption lang="Chinese (Traditional)" code="zh-Hant">, 'en': <Caption lang="English" code="en">, 'a.en': <Caption lang="English (auto-generated)" code="a.en">, 'fr': <Caption lang="French" code="fr">, 'de': <Caption lang="German" code="de">, 'hi': <Caption lang="Hindi" code="hi">, 'id': <Caption lang="Indonesian" code="id">, 'it': <Caption lang="Italian" code="it">, 'ja': <Caption lang="Japanese" code="ja">, 'ko': <Caption lang="Korean" code="ko">, 'mr': <Caption lang="Marathi" code="mr">, 'pl': <Caption lang="Polish" code="pl">, 'pt': <Caption lang="Portuguese" code="pt">, 'pa': <Caption lang="Punjabi" code="pa">, 'ru': <Caption lang="Russian" code="ru">, 'es': <Caption lang="Spanish" code="es">, 'ta': <Caption lang="Tamil" code="ta">, 'te': <Caption lang="Telugu" code="te">, 'th': <Caption lang="Thai" code="th">, 'tr': <Caption lang="Turkish" code="tr">, 'vi': <Caption lang="Vietnamese" code

In [9]:
cap.splitlines()

['1',
 '00:00:00,080 --> 00:00:05,160',
 'i am currently on an island infested',
 '',
 '2',
 '00:00:02,159 --> 00:00:08,240',
 "with space but I'm visiting way deadlier",
 '',
 '3',
 '00:00:05,160 --> 00:00:10,000',
 'places like Scalia Mountain covered in',
 '',
 '4',
 '00:00:08,240 --> 00:00:11,920',
 "ice i'm going to die let me down let me",
 '',
 '5',
 '00:00:10,000 --> 00:00:14,559',
 'down driving a road that has killed',
 '',
 '6',
 '00:00:11,920 --> 00:00:16,880',
 "thousands of people we're like 7 in from",
 '',
 '7',
 '00:00:14,559 --> 00:00:18,279',
 'falling to our death swimming unaged',
 '',
 '8',
 '00:00:16,880 --> 00:00:21,520',
 'with vicious',
 '',
 '9',
 '00:00:18,279 --> 00:00:23,600',
 'sharks and so much more all starting in',
 '',
 '10',
 '00:00:21,520 --> 00:00:25,680',
 'this cage in the middle of an African',
 '',
 '11',
 '00:00:23,600 --> 00:00:27,680',
 'safari where there are lions cheetahs',
 '',
 '12',
 '00:00:25,680 --> 00:00:29,119',
 'hippos and a doz

In [10]:
yt = YouTube(url)

# Get all video-only streams
streams = yt.streams.filter(res="1080p").first()
# video_streams = yt.streams.filter(only_video=True)

print("Available Resolutions:")
# for stream in video_streams:
print(f"Resolution: {streams.resolution}, Format: {streams.mime_type}")

Available Resolutions:
Resolution: 1080p, Format: video/mp4


In [ ]:
# Download the video with the title as the filename
# logging.info(f"Downloading: {yt.title} (Quality: {video.resolution}, Size: {video.filesize_mb:.1f}MB)")
# video = yt.streams.filter(res="720p").first()
# video_path = video.download(output_path=temp_dir)
video_path = streams.download(output_path=temp_dir)

logging.info(f"Video downloaded to {video_path}")

In [ ]:
streams

In [ ]:

    try:
        
        # return video_path
    except Exception as e:
        if "HTTP Error 400" in str(e):
            raise ValueError(f"YouTube rejected the request. The video might be age-restricted, private, or unavailable in your region. Try a different video.")
        elif "Video unavailable" in str(e):
            raise ValueError(f"This video is unavailable. It might have been removed or set to private.")
        elif "This video is age restricted" in str(e):
            raise ValueError(f"This video is age-restricted and cannot be downloaded.")
        else:
            raise ValueError(f"Failed to download YouTube video: {str(e)}")
        
except Exception as e:
    logging.error(f"Error downloading YouTube video: {str(e)}")
    raise ValueError(f"Failed to download YouTube video: {str(e)}")

In [16]:
lines = cap.splitlines()
dur = lines[1]
dur

'00:00:00,080 --> 00:00:05,160'

In [19]:
st = dur.split('-->')[0].strip()
end = dur.split('-->')[1].strip()
st, end


('00:00:00,080', '00:00:05,160')

In [ ]:
import moviepy